In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
#!conda install -c conda-forge geopy --yes
import geocoder

In [75]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [76]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [77]:
rows = table.find_all('tr')

In [78]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [79]:

df_ankara = pd.DataFrame(columns = columns)

In [80]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [81]:

df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [82]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [83]:

districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [84]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [85]:
df_ankara_loc.drop(columns="Population (2017)", axis=1, inplace=True)

In [86]:
df_ankara_loc.drop(columns="Density (per km²)", axis=1, inplace=True)

In [87]:
df_ankara_loc.drop(columns="Area (km²)", axis=1, inplace=True)

In [88]:
df_ankara_loc.head()

,District,Latitude,Longitude
0,Akyurt,40.13082,33.08719
1,Altındağ,39.94171,32.85445
2,Ayaş,40.01516,32.33270
3,Bala,39.55391,33.12352
4,Beypazarı,40.16811,31.92052


In [89]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print("Libraries imported")

Libraries imported


In [90]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207893, 32.8540412.


In [91]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [92]:
CLIENT_ID = "FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ"
CLIENT_SECRET = "OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC" 
VERSION = "20180605"

In [93]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [94]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [95]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ&client_secret=OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC&v=20180605&ll=40.13082000000003,33.08719000000008&radius=500&limit=100'

In [96]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ac3a450b3b6117fb053b9e'},
 'response': {'headerLocation': 'Akyurt',
  'headerFullLocation': 'Akyurt, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 40.13532000450003,
    'lng': 33.093064640939495},
   'sw': {'lat': 40.126319995500026, 'lng': 33.08131535906066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5049c781e4b0a7a32d840eb1',
       'name': 'Meşhur Köfteci Bodur',
       'location': {'address': 'Cumhuriyet Meydani',
        'crossStreet': 'Cumhuriyet Meydani',
        'lat': 40.13071902888477,
        'lng': 33.08677622815274,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.13071902888477,
          'lng': 33.08677622815274}],
        '

In [97]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [98]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
4,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492
7,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709
8,Akyurt,Farm,40.131012,33.083330
9,Benliler Supermarket,Convenience Store,40.130252,33.081423


In [99]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [102]:
print(ankara_venues.shape)
ankara_venues.head(250)

(1679, 7)


,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132672,33.084306,Kebab Restaurant
1,Akyurt,40.13082,33.08719,Akyurt kültür parkı,40.133123,33.085071,Park
2,Akyurt,40.13082,33.08719,Meşhur Köfteci Bodur,40.130719,33.086776,Turkish Restaurant
3,Akyurt,40.13082,33.08719,Onur pide ve kebap salonu,40.130807,33.086422,Kebab Restaurant
4,Akyurt,40.13082,33.08719,Beyazıt Sofrası,40.130542,33.081901,Steakhouse
5,Akyurt,40.13082,33.08719,CrossFit Center 34,40.124721,33.076126,Gym
6,Akyurt,40.13082,33.08719,Yurtalan,40.132743,33.084385,Turkish Restaurant
7,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132641,33.084492,Doner Restaurant
8,Akyurt,40.13082,33.08719,Benliler Supermarket,40.130252,33.081423,Convenience Store
9,Akyurt,40.13082,33.08719,Akyurt,40.131012,33.083330,Farm


In [103]:
ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,26,26,26,26,26,26
Altındağ,100,100,100,100,100,100
Ayaş,37,37,37,37,37,37
Bala,12,12,12,12,12,12
Beypazarı,66,66,66,66,66,66
Elmadağ,49,49,49,49,49,49
Etimesgut,100,100,100,100,100,100
Evren,5,5,5,5,5,5
Gölbaşı,100,100,100,100,100,100


In [104]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 233 unique venue categories.


In [105]:
ankara_onehot.shape

(1679, 234)

In [106]:

ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport Lounge,Airport Service,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Bookstore,Borek Place,Botanical Garden,Boutique,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Castle,Cigkofte Place,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Courthouse,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Dive Bar,Dive Shop,Dog Run,Doner Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indie Theater,Insurance Office,Internet Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Kumru Restaurant,Kuruyemişçi,Lake,Light Rail Station,Liquor Store,Lounge,Manti Place,Market,Mediterranean Restaurant,Men's Store,Meyhane,Middle Eastern Restaurant,Military Base,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,National Park,Nature Preserve,Night Market,Nightclub,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Lab,Pide Place,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Tourist Information Center,Track,Trail,Tree,Turkish Coffeehouse,Turkish Home Cooking Restaurant,Turkish Restaurant,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.00,0.00,0.000000,0.038462,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.038462,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.038462,0.000000,0.000000,0.000000,0.00000,0.00000,0.153846,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.038462,0.000000,0.038462,0.000000,0.00000,0.00,0.000000,0.0000

In [107]:
ankara_grouped.shape

(27, 234)

In [108]:

num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Farm  0.15
1  Turkish Restaurant  0.12
2    Kebab Restaurant  0.08
3                Park  0.08
4              Garden  0.04
5            Mountain  0.04
6          Steakhouse  0.04
7                 Gym  0.04
8          Bagel Shop  0.04
9                Café  0.04


----Altındağ----
                venue  freq
0                Café  0.11
1               Hotel  0.07
2      History Museum  0.07
3        Antique Shop  0.05
4  Turkish Restaurant  0.05
5             Theater  0.04
6       Jewelry Store  0.04
7       Historic Site  0.04
8          Restaurant  0.03
9         Art Gallery  0.03


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.14
1               Restaurant  0.08
2         Kebab Restaurant  0.05
3            Auto Workshop  0.05
4                     Lake  0.05
5                 Tea Room  0.05
6                     Café  0.05
7                     Park  0.05
8               Food Court  0.03
9  Fr

                  venue  freq
0                  Café  0.12
1                 Plaza  0.07
2    Turkish Restaurant  0.07
3    Seafood Restaurant  0.05
4      Kebab Restaurant  0.05
5  Fast Food Restaurant  0.05
6            Restaurant  0.05
7           Gas Station  0.02
8            Steakhouse  0.02
9           Beer Garden  0.02




In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [110]:

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(27)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Park,Kebab Restaurant,Big Box Store,Doner Restaurant,Mountain,Basketball Court,Furniture / Home Store,Steakhouse
1,Altındağ,Café,Hotel,History Museum,Antique Shop,Turkish Restaurant,Historic Site,Jewelry Store,Theater,Restaurant,Steakhouse
2,Ayaş,Turkish Restaurant,Restaurant,Park,Kebab Restaurant,Tea Room,Lake,Auto Workshop,Café,Plaza,Breakfast Spot
3,Bala,Electronics Store,Big Box Store,Turkish Restaurant,Nature Preserve,Business Service,Café,Steakhouse,Bakery,Garden,Park
4,Beypazarı,Turkish Restaurant,History Museum,Hotel,Tea Room,Motel,Scenic Lookout,Bakery,Park,Fast Food Restaurant,Restaurant
5,Elmadağ,Café,Park,Campground,Dessert Shop,Kebab Restaurant,Convenience Store,Bar,Meyhane,Kofte Place,Forest
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Arcade,Gym,Steakhouse,Dessert Shop,Kebab Restaurant,Garden,Bakery
7,Evren,Park,Steakhouse,Waterfront,Grocery Store,Yoga Studio,Entertainment Service,Event Service,Event Space,Farm,Farmers Market
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Dessert Shop,Coffee Shop,Middle Eastern Restaurant,Arcade,Bakery
9,Güdül,Convenience Store,Department Store,Athletics & Sports,Diner,Castle,River,Turkish Restaurant,Botanical Garden,Plaza,Souvenir Shop


In [115]:
!pip install kmeans
from sklearn.cluster import KMeans

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for kmeans: filename=kmeans-1.0.2-cp37-cp37m-linux_x86_64.whl size=12361 sha256=d4e5e470c754b063e1ab503c80fdf3d07676cbc6130a792d50aaf36c5f069cf8
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/4a/26/51/688c6987fdc6a10c6186c3817f2ef7c92c50c078a900525c0e
Successfully built kmeans


In [118]:
ankara_merged = df_ankara
ankara_merged['Cluster Labels'] = kmeans.labels_

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

NameError: name 'kmeans' is not defined

In [119]:
ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([0, 0, 0, 0, 0, 0, 2, 0, 1], dtype=int32)

In [120]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

In [121]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 0, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

KeyError: 'Cluster Labels'

In [122]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 1, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

KeyError: 'Cluster Labels'

In [123]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 2, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

KeyError: 'Cluster Labels'

In [124]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 3, ankara_merged.columns[[0] + list(range(5, ankara_merged.shape[1]))]]

KeyError: 'Cluster Labels'